In [41]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import string

import regex as re
import pandas as pd
from sklearn.utils import shuffle
from nltk import LancasterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import movie_reviews, stopwords
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/faculty/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/faculty/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
reviews = pd.read_csv('/project/weaklabel_sampledata.csv', header=0, encoding='unicode_escape')
reviews.head()

,Freshness,Review
0,0,Parental Content Review
1,1,Director Wayne Wang proves with Maid in Manha...
2,0,the lack of dramatic development doesn't leav...
3,1,A riveting documentary that explains educatio...
4,0,"The problem with The Informant!, aside from t..."


In [6]:
reviews_list = reviews['Review'].astype(str).tolist()
reviews_string = ''
reviews_string = reviews_string.join(reviews_list).lower()
reviews_string = reviews_string.replace('"', '').replace("'", '').replace('\n','').replace(',','').replace('[','').replace(']','')
tokens = reviews_string.split()

len(tokens)

107726

In [7]:
# remove punctuation from each token
table = str.maketrans('', '', string.punctuation)
tokens = [w.translate(table) for w in tokens]

# remove remaining tokens that are not alphabetic
tokens = [word for word in tokens if word.isalpha()]

# filter out stop words
stop_words = set(stopwords.words('english'))
tokens = [w for w in tokens if not w in stop_words]

# filter out short tokens
tokens = [word for word in tokens if len(word) > 1]

len(tokens)

60915

In [8]:
from operator import itemgetter
from collections import Counter

# Count how many times each word appears
count = Counter(tokens).items()
sorted_count = sorted(count, key = itemgetter(1))
sorted_count.reverse()

# Select 5000 most frequent words
top5000 = [i[0] for i in sorted_count[:5000]]
sorted_count

[('film', 774),
 ('movie', 563),
 ('one', 362),
 ('like', 342),
 ('story', 271),
 ('much', 225),
 ('even', 207),
 ('good', 200),
 ('films', 180),
 ('doesnt', 176),
 ('make', 175),
 ('never', 165),
 ('characters', 154),
 ('comedy', 151),
 ('way', 147),
 ('time', 142),
 ('enough', 140),
 ('movies', 138),
 ('little', 135),
 ('thats', 131),
 ('theres', 131),
 ('best', 131),
 ('makes', 130),
 ('full', 129),
 ('action', 128),
 ('director', 128),
 ('funny', 125),
 ('life', 124),
 ('great', 122),
 ('first', 121),
 ('get', 121),
 ('would', 120),
 ('feels', 113),
 ('well', 112),
 ('fun', 112),
 ('made', 112),
 ('could', 112),
 ('may', 111),
 ('love', 110),
 ('us', 107),
 ('two', 107),
 ('better', 107),
 ('performances', 101),
 ('every', 101),
 ('might', 101),
 ('review', 101),
 ('bad', 99),
 ('work', 99),
 ('dont', 98),
 ('still', 97),
 ('isnt', 95),
 ('many', 94),
 ('performance', 92),
 ('new', 91),
 ('audience', 90),
 ('also', 88),
 ('really', 88),
 ('see', 88),
 ('cast', 88),
 ('character', 8

In [9]:
from nltk import LancasterStemmer

# initialize Lancaster Stemmer
LS = LancasterStemmer()
lemmatized = []
for l in tokens: lemmatized.append(LS.stem(l))

# Count how many times each word appears
count = Counter(lemmatized).items()
sorted_count = sorted(count, key = itemgetter(1))
sorted_count.reverse()

# Select 5000 most frequent words
top5000 = [i[0] for i in sorted_count[:5000]]
sorted_count

[('film', 968),
 ('movy', 701),
 ('lik', 389),
 ('on', 381),
 ('mak', 366),
 ('act', 353),
 ('ev', 321),
 ('story', 298),
 ('charact', 251),
 ('feel', 237),
 ('real', 227),
 ('much', 225),
 ('tim', 216),
 ('direct', 214),
 ('perform', 207),
 ('good', 206),
 ('get', 196),
 ('com', 191),
 ('tak', 178),
 ('doesnt', 176),
 ('us', 175),
 ('way', 172),
 ('comedy', 166),
 ('nev', 165),
 ('work', 164),
 ('man', 157),
 ('end', 152),
 ('ful', 149),
 ('look', 145),
 ('watch', 143),
 ('enough', 140),
 ('lov', 140),
 ('giv', 139),
 ('gre', 136),
 ('littl', 135),
 ('funny', 134),
 ('best', 133),
 ('ther', 132),
 ('that', 131),
 ('fin', 128),
 ('mad', 127),
 ('lif', 127),
 ('entertain', 126),
 ('thing', 125),
 ('play', 124),
 ('first', 123),
 ('hum', 123),
 ('audy', 121),
 ('would', 120),
 ('seem', 117),
 ('fun', 116),
 ('tal', 115),
 ('see', 114),
 ('wel', 114),
 ('bet', 114),
 ('vis', 113),
 ('could', 112),
 ('may', 111),
 ('sens', 107),
 ('bad', 107),
 ('two', 107),
 ('emot', 106),
 ('year', 106),

In [44]:
word_matrix = []

for i in a: word_matrix.append([1 if j in i else 0 for j in top5000])
features = pd.DataFrame(word_matrix, columns = top5000, index = reviews.index)
features['review']=reviews['Review']
features['freshness']=reviews['Freshness']
features

,film,movy,lik,on,mak,act,ev,story,charact,feel,...,showbo,scrappy,withinsweaty,minorkey,glasss,medy,yel,denzel,behemo,freshness
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4997,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [54]:
a=[]
for i in reviews['Review'] :
    i = i.lower()
    i = word_tokenize(i)
    for j in i:
        j = LS.stem(j)
        
    a.append(i)
a

[['parental', 'content', 'review'],
 ['director',
  'wayne',
  'wang',
  'proves',
  'with',
  'maid',
  'in',
  'manhattan',
  'that',
  'even',
  'predictable',
  'movies',
  'can',
  'leave',
  'you',
  'feeling',
  'warm',
  'and',
  'fuzzy',
  '.'],
 ['the',
  'lack',
  'of',
  'dramatic',
  'development',
  'does',
  "n't",
  'leave',
  'much',
  'for',
  'the',
  'actors',
  'to',
  'work',
  'with'],
 ['a',
  'riveting',
  'documentary',
  'that',
  'explains',
  'education',
  'and',
  'the',
  'violent',
  'changes',
  'that',
  'are',
  'rocking',
  'pakistan',
  '.'],
 ['the',
  'problem',
  'with',
  'the',
  'informant',
  '!',
  ',',
  'aside',
  'from',
  'that',
  'overcoaxing',
  'exclamation',
  'mark',
  ',',
  'is',
  'that',
  'it',
  "'s",
  'not',
  'especially',
  'funny',
  '.'],
 ['too',
  'tepid',
  'and',
  'too',
  'predictable',
  'of',
  'a',
  'picture',
  'to',
  'simply',
  'sail',
  'away',
  'into',
  'the',
  'sunset',
  '.'],
 ['bernie',
  'lacks'

In [39]:
reviews['Review'].tolist()

[' Parental Content Review',
 ' Director Wayne Wang proves with Maid in Manhattan that even predictable movies can leave you feeling warm and fuzzy.',
 " the lack of dramatic development doesn't leave much for the actors to work with",
 ' A riveting documentary that explains education and the violent changes that are rocking Pakistan.',
 " The problem with The Informant!, aside from that overcoaxing exclamation mark, is that it's not especially funny.",
 ' Too tepid and too predictable of a picture to simply sail away into the sunset.',
 ' Bernie lacks the edge of malice one might reasonably expect from so nasty a tale.',
 " Simply one of the poorest entertainments ever offered up to the world's children.",
 ' More drab "fi" than dreamy "sci."',
 ' With its emphasis on nature and its mudbound farmers, this beautifully crafted film recalls other earthly European historical dramas set in an isolated countryside.',
 " It's not like having a real film of Nijinsky, but at least it's better 